In [ ]:
# @title تشغيل سكربت SiFThon - الإصدار المقاوم 

import os
import requests
import zipfile
import subprocess
import sys
import socket
import time
import json
import threading
from pathlib import Path

class SiFThonAutoRestart:
    def __init__(self):
        self.script_path = None
        self.env_file = "SiFThon/.env"
        self.config_file = "sifthon_config.json"
        self.current_version = "2.0"
        self.process = None
        self.restart_count = 0
        self.max_restarts = 100

    def verify_environment(self):
        """التحقق من بيئة Google Colab"""
        print("جاري التحقق من البيئة...")

        checks = {
            'colab_environment': False,
            'google_domain': False,
            'required_libraries': False
        }

        if 'COLAB_GPU' in os.environ or 'COLAB_JUPYTER_TOKEN' in os.environ:
            checks['colab_environment'] = True

        try:
            hostname = socket.gethostname()
            if 'colab' in hostname.lower():
                checks['google_domain'] = True
        except:
            pass

        try:
            import google.colab
            checks['required_libraries'] = True
        except ImportError:
            pass

        print(f"نتائج التحقق: {checks}")

        if not any(checks.values()):
            print("خطأ: هذا السكربت يعمل فقط على Google Colab")
            sys.exit(1)

        return True

    def load_config(self):
        """تحميل الإعدادات المحفوظة"""
        if os.path.exists(self.config_file):
            try:
                with open(self.config_file, 'r') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def save_config(self, config):
        """حفظ الإعدادات"""
        try:
            with open(self.config_file, 'w') as f:
                json.dump(config, f, indent=4)
        except:
            pass

    def download_sifthon(self):
        """تحميل سكربت SiFThon"""
        url = 'http://especi1.serv00.net/url.php'

        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36',
            'Accept': 'application/zip, /',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,ar;q=0.8',
            'Referer': 'https://colab.research.google.com/',
            'Origin': 'https://colab.research.google.com'
        }

        print("جاري الاتصال بالسيرفر لتحميل أحدث إصدار...")

        try:
            response = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
            print(f"كود الاستجابة: {response.status_code}")

            if response.status_code == 200:
                with open('SiFThon.zip', 'wb') as f:
                    f.write(response.content)
                print("تم تحميل أحدث إصدار بنجاح")
                return True
            else:
                print(f"فشل التحميل: {response.status_code}")
                return False

        except requests.exceptions.RequestException as e:
            print(f"خطأ في الاتصال: {e}")
            return False

    def extract_and_setup(self):
        """فك الضغط والإعداد"""
        try:
            print("جاري فك الضغط...")
            with zipfile.ZipFile('SiFThon.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            print("تم فك الضغط بنجاح")

            # البحث عن مجلد SiFThon
            for root, dirs, files in os.walk('.'):
                if 'SiFThon' in dirs:
                    self.script_path = os.path.join(root, 'SiFThon')
                    break

            if not self.script_path:
                print("خطأ: لم يتم العثور على مجلد SiFThon")
                return False

            os.chdir(os.path.dirname(self.script_path))
            return True

        except zipfile.BadZipFile:
            print("خطأ: ملف ZIP تالف")
            return False

    def install_requirements(self):
        """تثبيت المكتبات المطلوبة"""
        print("جاري تثبيت المكتبات المطلوبة...")
        requirements = [
            'telethon', 'pyrogram', 'requests', 'tabulate',
            'ipaddress', 'telegraph', 'python-decouple',
            'Pillow', 'pytz', 'gitpython', 'fpdf', 
            'youtubesearchpython', 'yt-dlp'
        ]

        for req in requirements:
            try:
                subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', req], 
                             check=True, capture_output=True)
                print(f"✓ تم تثبيت: {req}")
            except subprocess.CalledProcessError:
                print(f"✗ فشل تثبيت: {req}")

    def get_credentials(self):
        """الحصول على بيانات الاعتماد"""
        config = self.load_config()
        
        if config.get('saved_credentials') and config.get('installation_type') == 'old':
            print("\nتم العثور على إعدادات سابقة!")
            choice = input("هل تريد استخدام الإعدادات السابقة؟ (نعم/لا): ").strip().lower()
            
            if choice in ['نعم', 'yes', 'y', '']:
                return config['saved_credentials']

        print("\nأدخل بيانات الاتصال الجديدة:")
        
        API_ID = input("API_ID: ").strip()
        API_HASH = input("API_HASH: ").strip()
        SESSION = input("SESSION: ").strip()
        BOT_TOKEN = input("BOT_TOKEN: ").strip()

        credentials = {
            'API_ID': API_ID,
            'API_HASH': API_HASH,
            'SESSION': SESSION,
            'BOT_TOKEN': BOT_TOKEN
        }

        # حفظ الإعدادات
        config['saved_credentials'] = credentials
        config['installation_type'] = 'old'
        config['version'] = self.current_version
        self.save_config(config)

        return credentials

    def create_env_file(self, credentials):
        """إنشاء ملف البيئة"""
        env_content = f"""API_ID={credentials['API_ID']}
API_HASH={credentials['API_HASH']}
SESSION={credentials['SESSION']}
BOT_TOKEN={credentials['BOT_TOKEN']}
"""

        os.makedirs('SiFThon', exist_ok=True)
        with open(self.env_file, 'w') as f:
            f.write(env_content)

        print("\n✓ تم حفظ بيانات الاتصال في ملف .env")

    def monitor_process(self):
        """مراقبة العملية وإعادة التشغيل التلقائي"""
        while self.restart_count < self.max_restarts:
            try:
                print(f"\n{'='*50}")
                print(f"بدء التشغيل - المحاولة #{self.restart_count + 1}")
                print(f"{'='*50}")

                self.process = subprocess.Popen([sys.executable, '-m', 'SiFThon'],
                                             stdout=subprocess.PIPE,
                                             stderr=subprocess.STDOUT,
                                             text=True,
                                             bufsize=1,
                                             universal_newlines=True)

                # قراءة المخرجات في الوقت الحقيقي
                while True:
                    output = self.process.stdout.readline()
                    if output == '' and self.process.poll() is not None:
                        break
                    if output:
                        print(output.strip())

                return_code = self.process.poll()
                
                if return_code == 0:
                    print("\n✓ السكربت انتهى بشكل طبيعي")
                    break
                else:
                    self.restart_count += 1
                    print(f"\n⚠ السكربت توقف. إعادة التشغيل خلال 5 ثوان... ({self.restart_count}/{self.max_restarts})")
                    
                    # تحديث السكربت كل 5 إعادة تشغيل
                    if self.restart_count % 5 == 0:
                        print(" جاري تحديث السكربت إلى أحدث إصدار...")
                        self.update_script()
                    
                    time.sleep(5)

            except KeyboardInterrupt:
                print("\n تم إيقاف المراقبة يدوياً")
                if self.process:
                    self.process.terminate()
                break
            except Exception as e:
                print(f"\n خطأ في المراقبة: {e}")
                self.restart_count += 1
                time.sleep(5)

        if self.restart_count >= self.max_restarts:
            print("\n تم الوصول إلى الحد الأقصى لمحاولات إعادة التشغيل")

    def update_script(self):
        """تحديث السكربت إلى أحدث إصدار"""
        print("جاري التحقق من التحديثات...")
        
        # حفظ الإعدادات الحالية
        config = self.load_config()
        
        if self.download_sifthon():
            if self.extract_and_setup():
                print("✓ تم التحديث بنجاح")
                # استعادة الإعدادات
                if config.get('saved_credentials'):
                    self.create_env_file(config['saved_credentials'])
            else:
                print("✗ فشل في التحديث")

    def main(self):
        """الدالة الرئيسية"""
        print("=" * 60)
        print(" سكربت SiFThon - الإصدار المقاوم ")
        print(" يعيد التشغيل تلقائياً عند التوقف")
        print(" يتم التحديث تلقائياً كل 5 إعادة تشغيل")
        print("=" * 60)

        # التحقق من البيئة
        if not self.verify_environment():
            return

        # التحقق من نوع التثبيت
        config = self.load_config()
        
        if config.get('installation_type'):
            print("\nتم العثور على تثبيت سابق!")
            choice = input("هل تريد تثبيت جديد أم استخدام القديم؟ (جديد/قديم): ").strip().lower()
            
            if choice in ['جديد', 'new', 'n']:
                config['installation_type'] = 'new'
                self.save_config(config)
                print("سيتم تنصيب نسخة جديدة...")
            else:
                print("سيتم استخدام الإعدادات القديمة...")

        # تحميل السكربت
        if not self.download_sifthon():
            print("فشل في تحميل السكربت")
            return

        # فك الضغط
        if not self.extract_and_setup():
            return

        # تثبيت المتطلبات
        self.install_requirements()

        # الحصول على بيانات الاعتماد
        credentials = self.get_credentials()

        # إنشاء ملف البيئة
        self.create_env_file(credentials)

        print("\n" + "=" * 50)
        print(" بدء التشغيل مع المراقبة المستمرة...")
        print(" يمكنك إيقاف التشغيل بالضغط على Ctrl+C")
        print("=" * 50 + "\n")

        # بدء المراقبة
        self.monitor_process()

        print("\n" + "=" * 50)
        print("انتهى تشغيل سكربت SiFThon")
        print("=" * 50)

# تشغيل السكربت
if __name__ == "__main__":
    sifthon = SiFThonAutoRestart()
    sifthon.main()

---
##  ميزات الإصدار المقاوم

###  إعادة التشغيل التلقائي
- يعيد التشغيل تلقائياً عند توقف السكربت
- حد أقصى 100 محاولة إعادة تشغيل
- تأخير 5 ثواني بين المحاولات

###  حفظ الإعدادات
- يحفظ بيانات API تلقائياً
- يسأل عن استخدام الإعدادات القديمة أو الجديدة
- يحفظ نوع التثبيت (جديد/قديم)

###  التحديث التلقائي
- يتم التحديث كل 5 إعادة تشغيل
- يحمل أحدث إصدار من السيرفر
- يحافظ على الإعدادات المحفوظة

###  المراقبة المستمرة
- يعرض المخرجات في الوقت الحقيقي
- يظهر عدد محاولات إعادة التشغيل
- يمكن إيقافه يدوياً بـ Ctrl+C

---
## ⚙️ آلية العمل

1. **التحقق من البيئة** → يتأكد من تشغيله على Colab
2. **فحص الإعدادات** → يتحقق من وجود تثبيت سابق
3. **تحميل السكربت** → يحمل أحدث إصدار من السيرفر
4. **تثبيت المتطلبات** → يثبت جميع المكتبات اللازمة
5. **إدخال البيانات** → يطلب بيانات API أو يستخدم المحفوظة
6. **المراقبة المستمرة** → يشغل السكربت ويراقبه باستمرار
7. **الإعادة التلقائية** → يعيد التشغيل عند التوقف

---
##  خيارات التثبيت

### التثبيت الجديد
- يحمل أحدث إصدار
- يطلب بيانات API جديدة
- يحفظ الإعدادات للمستقبل

###  استخدام القديم
- يستخدم الإعدادات المحفوظة
- لا يطلب بيانات API مرة أخرى
- يحمل أحدث إصدار مع الحفاظ على الإعدادات

---
<h6 align="center"> SiFThon - الإصدار المقاوم </h6>